In [1]:
%load_ext autoreload
%autoreload 2

from jupyter_cadquery import set_defaults, show, Camera, Collapse
import math
from build123d import *

set_defaults(
	viewer="Sidecar",
	edge_accuracy=0.0001,
	ticks=25,
	grid=[True, True, True],
	cad_width=780,
	axes=True,
	axes0=True,
	glass=True,
	orbit_control=True,
	reset_camera=Camera.CENTER,
	collapse=Collapse.LEAVES
)

Overwriting auto display for build123d BuildPart, BuildSketch, BuildLine, ShapeList


In [2]:
TABLET_THICKNESS = 5
CLAMP_THICKNESS = 1
CLAMP_WIDTH  = 40

RAIL_WIDTH = 30
RAIL_HEIGHT = 6
TABLET_WIDTH = 140
THUMB_SCREW_DIAM = 30
WASHER_THICKNESS = 0.5

MARGIN = 0.2

In [3]:
SCREW_SHAFT_DIAM = 5
SCREW_HEAD_DIAM = 8.6
SCREW_HEAD_LENGTH = 5
SCREW_SHAFT_LENGTH = 25
HEX_NUT_THICKNESS = SCREW_HEAD_LENGTH / 2

In [30]:
STEP = RAIL_HEIGHT / 2

def make_rail_profile(width_step, height_step):
    return Polyline(
        (width_step, 0),
        (width_step + height_step, height_step),
        (width_step, height_step * 2),
        (-width_step, height_step * 2),
        (-(width_step + height_step), height_step),
        (-width_step, 0)
    ).close()

def make_rider_profile(width_step, height_step, margin=MARGIN):
    return Polyline(
        (-(width_step + margin), height_step * 2),
        (-(width_step + height_step + margin), height_step),
        (-(width_step + margin), 0),
        (width_step + margin, 0),
        (width_step + height_step + margin, height_step),
        (width_step + margin, height_step * 2),
        (width_step + margin + height_step * 2, height_step * 2),
        (width_step + margin + height_step * 2, 0),
        (width_step + margin + height_step * 2, -height_step),
        (-(width_step + margin + height_step * 2), -height_step),
        (-(width_step + margin + height_step * 2), height_step * 2),
    ).close()

def make_rail(
    rail_width,
    rail_height,
    rail_length,
    thumb_screw_radius,
    screw_head_radius,
    screw_shaft_radius,
    hex_nut_thickness
):
    profile = make_rail_profile(rail_width / 2, rail_height / 2)
    rail = Plane.XZ * extrude(Face(profile), rail_length)

    rail -= Plane(
        rail.faces().filter_by(Axis.Z).sort_by(Axis.Z).first
    ) * Pos(
        0,
        rail_length / 2 - thumb_screw_radius
    ) * Hole(
        screw_shaft_radius,
        rail_height
    )

    rail -= Plane(
        rail.faces().filter_by(Axis.Z).sort_by(Axis.Z).last
    ) * Pos(
        0,
        rail_length / 2 - thumb_screw_radius
    ) * extrude(
        RegularPolygon(screw_head_radius, 6),
        -hex_nut_thickness
    )
    # get screw_position/plane from rail
    return rail

def make_rider(
    rail_width,
    rail_height,
    rail_length,
    thumb_screw_radius,
    screw_shaft_radius
):
    profile = make_rider_profile(rail_width / 2, rail_height / 2)
    rider = Plane.XZ * extrude(-Face(profile), rail_length)

    p = Plane(
        rider.faces().filter_by(Axis.Z).sort_by(Axis.Z).first
    )
    rider = rider - extrude(
        p * SlotCenterPoint(
            (0,0),
            (0, rail_length - thumb_screw_radius),
            screw_shaft_radius
        ),
        rail_height
    )
    return rider

def make_thumbscrew(
    thumb_screw_radius,
    screw_shaft_radius,
    screw_shaft_length,
    screw_head_radius,
    screw_head_length,
    rider_height,
    washer_thickness
):
    thumbscrew = Cylinder(
        thumb_screw_radius,
        screw_shaft_length + screw_head_length - rider_height - washer_thickness,
        align=(Align.CENTER, Align.CENTER, Align.MIN)
    )
    thumbscrew -= Plane(
        thumbscrew.faces().filter_by(Axis.Z).first
    ) * CounterBoreHole(
        screw_shaft_radius,
        screw_head_radius,
        screw_head_length,
        screw_shaft_length
    )
    return thumbscrew

thumbscrew = make_thumbscrew(
    THUMB_SCREW_DIAM / 2,
    SCREW_SHAFT_DIAM / 2,
    SCREW_SHAFT_LENGTH,
    SCREW_HEAD_DIAM / 2,
    SCREW_HEAD_LENGTH,
    STEP * 3,
    WASHER_THICKNESS
)
rail, rider = make_rail(
    RAIL_WIDTH,
    RAIL_HEIGHT,
    TABLET_WIDTH / 2,
    THUMB_SCREW_DIAM / 2,
    SCREW_HEAD_DIAM / 2,
    SCREW_SHAFT_DIAM / 2,
    SCREW_HEAD_LENGTH / 2
), make_rider(
    RAIL_WIDTH,
    RAIL_HEIGHT,
    TABLET_WIDTH / 2,
    THUMB_SCREW_DIAM / 2,
    SCREW_SHAFT_DIAM / 2
)
bolt = make_bolt(
    SCREW_SHAFT_DIAM / 2,
    SCREW_SHAFT_LENGTH,
    SCREW_HEAD_DIAM / 2,
    SCREW_HEAD_LENGTH
)

p = Plane(rider.faces().filter_by(Axis.Z).sort_by(Axis.Z).first)
pos = p * Pos(0, -TABLET_WIDTH / 4 + THUMB_SCREW_DIAM / 2, WASHER_THICKNESS)

show(
    rail,
    rider,
    pos * thumbscrew,
    Plane(
        pos * thumbscrew.faces().filter_by(Axis.Z).first
    ).offset(-5) * bolt, colors=[
        (232, 176, 36),
        (232, 176, 36),
        (232, 176, 36),
        (150, 150, 150)
    ]
)

cc++


In [ ]:
class BoltBlank(BasePartObject):
    def __init__(self, shaft_radius, shaft_length, head_radius, head_length, rotation=(), mode=mode.ADD):
        super().__init(self, part, rotation, mode)